In [1]:
import numpy as np
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm

# --- Configuration ---
# Set device to GPU if available, otherwise CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Define paths to your data on Kaggle
FEATURES_PATH = '/kaggle/input/mse-landmarks/MSE_GCN_combined_features.npz'
JSON_PATH = '/kaggle/input/json-files/nslt_300.json'

# --- 1. Load Data from Files ---
print("Loading data files...")
# Load the combined landmark file. It's a large file, but np.load is memory-efficient.
features_data = np.load(FEATURES_PATH, allow_pickle=True)

# Load the JSON file to get splits and labels
with open(JSON_PATH, 'r') as f:
    json_data = json.load(f)
print("Data loading complete.")

# --- 2. Create Data Splits and Labels Dictionary ---
train_ids, val_ids, test_ids = [], [], []
labels_dict = {}

# We need to map the original class IDs (which can be non-sequential) to a zero-indexed range
# e.g., class IDs [191, 1, 43] -> [0, 1, 2]
unique_labels = sorted({info['action'][0] for info in json_data.values()})
class_to_idx = {label: i for i, label in enumerate(unique_labels)}
num_classes = len(unique_labels)

print(f"\nFound {num_classes} unique classes.")

for video_id, info in json_data.items():
    # Check if this video's features were successfully extracted and combined
    if video_id in features_data:
        # Get the original class label and map it to our new zero-indexed integer
        original_label = info['action'][0]
        labels_dict[video_id] = class_to_idx[original_label]
        
        # Add the video ID to the correct split list
        if info['subset'] == 'train':
            train_ids.append(video_id)
        elif info['subset'] == 'val':
            val_ids.append(video_id)
        elif info['subset'] == 'test':
            test_ids.append(video_id)

print(f"Training samples: {len(train_ids)}")
print(f"Validation samples: {len(val_ids)}")
print(f"Test samples: {len(test_ids)}")

Using device: cuda
Loading data files...
Data loading complete.

Found 300 unique classes.
Training samples: 3548
Validation samples: 900
Test samples: 668


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# --- Hyperparameters from the Paper ---
SEQ_LEN = 64 # Sequence length (T) 
BATCH_SIZE = 16 # Batch size 

class SignLanguageDataset(Dataset):
    """
    Corrected PyTorch Dataset. This version maps bone features to the joint dimension
    to make it compatible with the GCN architecture.
    """
    def __init__(self, video_ids, labels_dict, features_data, seq_len):
        self.video_ids = video_ids
        self.labels = labels_dict
        self.features_data = features_data
        self.seq_len = seq_len
        
        # Define the mapping from bone index to the end-joint index
        # This should match the order in your data extraction script's BONE_CONNECTIONS
        self.bone_to_joint_map = {
            0: 13, # LEFT_ELBOW
            1: 15, # LEFT_WRIST
            2: 14, # RIGHT_ELBOW
            3: 16, # RIGHT_WRIST
            4: 11  # LEFT_SHOULDER (from RIGHT_SHOULDER)
        }

    def __len__(self):
        return len(self.video_ids)

    def __getitem__(self, idx):
        video_id = self.video_ids[idx]
        
        data_dict = self.features_data[video_id].item()
        joint_seq = data_dict['joint_sequence'].astype(np.float32)
        bone_seq_flat = data_dict['bone_sequence'].astype(np.float32) # Shape: (T, 10)
        
        # --- Create V-dimensional Bone Tensor ---
        num_frames = bone_seq_flat.shape[0]
        num_joints = 65
        bone_features_per_bone = 2 # length and angle
        bone_seq = np.zeros((num_frames, num_joints, bone_features_per_bone), dtype=np.float32)

        for bone_idx, joint_idx in self.bone_to_joint_map.items():
            start_col = bone_idx * 2
            end_col = start_col + 2
            bone_seq[:, joint_idx, :] = bone_seq_flat[:, start_col:end_col]
        
        # --- Handle Padding/Truncating ---
        original_len = joint_seq.shape[0]
        if original_len < self.seq_len:
            pad_len = self.seq_len - original_len
            joint_seq = np.pad(joint_seq, ((0, pad_len), (0, 0), (0, 0)), 'constant')
            bone_seq = np.pad(bone_seq, ((0, pad_len), (0, 0), (0, 0)), 'constant')
        elif original_len > self.seq_len:
            joint_seq = joint_seq[:self.seq_len]
            bone_seq = bone_seq[:self.seq_len]
            
        label = self.labels[video_id]
        
        # Permute to (C, T, V)
        joint_seq_tensor = torch.tensor(joint_seq).permute(2, 0, 1) # Shape: (4, 64, 65)
        bone_seq_tensor = torch.tensor(bone_seq).permute(2, 0, 1)  # Shape: (2, 64, 65)

        return joint_seq_tensor, bone_seq_tensor, torch.tensor(label, dtype=torch.long)

# --- Instantiate Datasets ---
train_dataset = SignLanguageDataset(train_ids, labels_dict, features_data, SEQ_LEN)
val_dataset = SignLanguageDataset(val_ids, labels_dict, features_data, SEQ_LEN)
# We will create the test_dataset later during the evaluation phase

# --- Instantiate DataLoaders ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# --- Sanity Check ---
print("\n--- Running a sanity check on the DataLoader ---")
# Get one batch from the training loader
try:
    joint_batch, bone_batch, labels_batch = next(iter(train_loader))
    
    # Print the shapes to verify they are correct
    print(f"Joints batch shape: {joint_batch.shape}")
    print(f"Bones batch shape: {bone_batch.shape}")
    print(f"Labels batch shape: {labels_batch.shape}")

    print("\nSanity check passed! The data is shaped correctly.")
    print(f"Joints shape corresponds to (Batch Size, Features, Frames, Joints).")
    print(f"Bones shape corresponds to (Batch Size, Bone Features, Frames).")

except Exception as e:
    print(f"\nAn error occurred during the sanity check: {e}")


--- Running a sanity check on the DataLoader ---
Joints batch shape: torch.Size([16, 4, 64, 65])
Bones batch shape: torch.Size([16, 2, 64, 65])
Labels batch shape: torch.Size([16])

Sanity check passed! The data is shaped correctly.
Joints shape corresponds to (Batch Size, Features, Frames, Joints).
Bones shape corresponds to (Batch Size, Bone Features, Frames).


In [9]:
import torch
import torch.nn as nn
import numpy as np

class Graph:
    """
    The Graph to model the skeletons extracted by MediaPipe
    """
    def __init__(self, layout='mediapipe_65', strategy='uniform'):
        self.num_node = 65
        self.self_link = [(i, i) for i in range(self.num_node)]
        
        # Define connections between joints
        self.inward_bone_link = [
            (12, 14), (14, 16), (16, 18), (16, 20), (16, 22), (18, 20), # Right Arm
            (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (17, 19), # Left Arm
            (12, 11), # Shoulders
            (10, 8), (8, 6), (6, 5), (5, 4), (4, 0), # Face
            (9, 7), (7, 3), (3, 2), (2, 1), (1, 0), # Face
            # Connect hands to wrists
            (15, 23), (16, 44) 
        ]
        
        # Add hand connections (a simple chain for each finger)
        for hand_start_idx in [23, 44]: # 23=Left Hand, 44=Right Hand
            self.inward_bone_link.extend([(hand_start_idx, hand_start_idx + 1), (hand_start_idx + 1, hand_start_idx + 2),
                                          (hand_start_idx + 2, hand_start_idx + 3), (hand_start_idx + 3, hand_start_idx + 4)]) # Thumb
            for finger_start in range(5, 21, 4):
                 self.inward_bone_link.extend([(hand_start_idx, hand_start_idx + finger_start),
                                               (hand_start_idx + finger_start, hand_start_idx + finger_start + 1),
                                               (hand_start_idx + finger_start + 1, hand_start_idx + finger_start + 2),
                                               (hand_start_idx + finger_start + 2, hand_start_idx + finger_start + 3)])

        self.edge = self.self_link + self.inward_bone_link
        
        # Create adjacency matrix
        self.A = torch.zeros(self.num_node, self.num_node)
        for i, j in self.edge:
            self.A[j, i] = 1
            self.A[i, j] = 1

In [13]:
class MS_GCN(nn.Module):
    """
    Corrected Multi-Scale Graph Convolutional Network.
    This version uses a learnable, multi-subset adjacency matrix.
    """
    def __init__(self, in_channels, out_channels, A, num_subset=3):
        super().__init__()
        self.num_subset = num_subset
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels, out_channels, kernel_size=1) for _ in range(self.num_subset)
        ])
        
        # This creates a learnable parameter for the adjacency matrix of shape (K, V, V)
        # where K is the number of subsets.
        self.A = nn.Parameter(torch.tensor(np.stack([A.numpy()] * num_subset), dtype=torch.float32))

    def forward(self, x):
        # x shape: (N, C, T, V)
        N, C, T, V = x.shape
        
        y = None
        for i in range(self.num_subset):
            # Get the adjacency matrix for this subset, shape (V, V)
            A_sub = self.A[i]
            
            # Perform graph convolution (A * X)
            x_reshaped = x.view(N, C * T, V)
            z = torch.matmul(x_reshaped, A_sub).view(N, C, T, V)
            
            # Apply the channel-wise convolution (W)
            out = self.convs[i](z)
            
            # Aggregate results from subsets
            y = out + y if y is not None else out
            
        return y

class MS_TCN(nn.Module):
    # Multi-Scale Temporal Convolutional Network
    def __init__(self, in_channels, out_channels, kernel_size=5, stride=1, **kwargs):
        super().__init__()
        pad = (kernel_size - 1) // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=(kernel_size, 1),
                              padding=(pad, 0), stride=(stride, 1))
        self.bn = nn.BatchNorm2d(out_channels)
        self.swish = nn.SiLU() # SiLU is the Swish activation function

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.swish(x)

class MS_STGCN_Block_Component(nn.Module):
    # A single component block as shown in the paper's diagrams
    def __init__(self, in_channels, out_channels, A, stride=1, **kwargs):
        super().__init__()
        # The paper uses SandGlass layers, which are Depthwise -> Pointwise -> Pointwise -> Depthwise
        # We simplify this to a single Depthwise -> Pointwise (Separable Conv) for clarity.
        
        # Spatial Multi-Scale Module
        self.msgcn = MS_GCN(in_channels, out_channels, A)
        
        # Temporal Multi-Scale Module
        self.mstcn = MS_TCN(out_channels, out_channels, stride=stride)

    def forward(self, x):
        x = self.msgcn(x)
        x = self.mstcn(x)
        return x

In [14]:
class ST_JPA(nn.Module):
    """ Spatiotemporal Joints Part Attention """
    def __init__(self, in_channels, num_joints=65):
        super().__init__()
        
        # Define body parts as per the paper [cite: 284]
        self.part_indices = {
            'head': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            'left_arm': [11, 13, 15, 17, 19, 21],
            'right_arm': [12, 14, 16, 18, 20, 22],
            'left_hand': list(range(23, 44)),
            'right_hand': list(range(44, 65)),
        }
        
        # ST-Joint Attention components
        self.joint_att = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 4, kernel_size=1),
            nn.BatchNorm2d(in_channels // 4),
            nn.SiLU(),
            nn.Conv2d(in_channels // 4, in_channels, kernel_size=1)
        )
        
        # Part Attention components
        self.part_att = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 4, kernel_size=1),
            nn.BatchNorm2d(in_channels // 4),
            nn.SiLU(),
            nn.Conv2d(in_channels // 4, 5, kernel_size=1) # 5 parts
        )

    def forward(self, x):
        # x shape: (N, C, T, V)
        
        # --- ST-Joint Attention ---
        # Temporal pooling
        x_t = x.mean(dim=2, keepdim=True) # (N, C, 1, V)
        # Spatial pooling
        x_s = x.mean(dim=3, keepdim=True) # (N, C, T, 1)
        
        # The paper's algorithm is slightly ambiguous; we use a common fusion method.
        # Here we apply attention to spatial and temporal pooled features separately.
        attn_s = torch.sigmoid(self.joint_att(x_t)) # Spatial attention map (N, C, 1, V)
        attn_t = torch.sigmoid(self.joint_att(x_s)) # Temporal attention map (N, C, T, 1)
        
        stja_map = attn_t * attn_s # Broadcast to get (N, C, T, V)
        x_stja = x * stja_map
        
        # --- Part Attention ---
        pooled_x = x.mean(dim=2, keepdim=True) # (N, C, 1, V)
        part_scores = self.part_att(pooled_x) # (N, num_parts, 1, V)
        
        part_att_map = torch.zeros_like(part_scores[:, 0:1, ...]) # (N, 1, 1, V)
        for i, part_name in enumerate(self.part_indices):
            indices = self.part_indices[part_name]
            # Softmax over joints within each part
            part_map = F.softmax(part_scores[:, i:i+1, :, indices], dim=-1)
            part_att_map[:, :, :, indices] = part_map

        x_pa = x * part_att_map
        
        # --- Gated Fusion ---
        gate = torch.sigmoid(x_stja + x_pa)
        out = gate * x_stja + (1 - gate) * x_pa
        
        return out

In [4]:
import torch.nn.functional as F

class MS_STGCN_Block(nn.Module):
    def __init__(self, in_channels, out_channels, A, stride=1):
        super().__init__()
        self.gcn_tcn = MS_STGCN_Block_Component(in_channels, out_channels, A, stride=stride)
        self.jpa = ST_JPA(out_channels)
        self.residual = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride, 1)),
            nn.BatchNorm2d(out_channels)
        ) if in_channels != out_channels or stride != 1 else lambda x: x

    def forward(self, x):
        res = self.residual(x)
        x = self.gcn_tcn(x)
        x = self.jpa(x)
        return x + res

class MSE_GCN(nn.Module):
    def __init__(self, num_classes, num_joints=65, joint_in_channels=4, bone_in_channels=2):
        super().__init__()
        
        graph = Graph()
        A = graph.A
        
        # Input Normalization
        self.bn_joint = nn.BatchNorm2d(joint_in_channels)
        self.bn_bone = nn.BatchNorm2d(bone_in_channels)
        
        # --- Define Streams ---
        self.joint_stream = self._make_stream(joint_in_channels, A)
        self.bone_stream = self._make_stream(bone_in_channels, A)
        
        # --- Post-Fusion Layers ---
        self.stage3 = MS_STGCN_Block(48*2, 64, A, stride=2) # 48 from each stream
        self.stage4 = MS_STGCN_Block(64, 128, A, stride=2)
        
        # --- Prediction Head ---
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(128, num_classes)

    def _make_stream(self, in_channels, A):
        return nn.Sequential(
            # Basic Block (Stage 0) as per paper's description
            MS_STGCN_Block_Component(in_channels, 64, A),
            MS_STGCN_Block(64, 48, A), # Stage 1
            MS_STGCN_Block(48, 48, A)  # Stage 2
        )

    def forward(self, x_joint, x_bone):
        # Normalize inputs
        x_joint = self.bn_joint(x_joint)
        x_bone = self.bn_bone(x_bone)
        
        # Process streams in parallel
        out_joint = self.joint_stream(x_joint)
        out_bone = self.bone_stream(x_bone)
        
        # Early Fusion (Concatenate after Stage 2)
        out = torch.cat([out_joint, out_bone], dim=1)
        
        # Post-Fusion
        out = self.stage3(out)
        out = self.stage4(out)
        
        # Prediction
        out = self.pool(out)
        out = out.view(out.size(0), -1) # Flatten
        out = self.dropout(out)
        out = self.fc(out)
        
        return out

# --- Instantiate the final model ---
model = MSE_GCN(num_classes=num_classes).to(DEVICE)
print("\nModel Architecture Created Successfully!")

# Print a summary of the model
from torchsummary import summary
# We need to provide dummy inputs matching the model's expected shapes
summary(model, [(4, SEQ_LEN, 65), (2, SEQ_LEN, 65)])

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

# --- Hyperparameters from the Paper ---
EPOCHS = 350
LEARNING_RATE = 0.1
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9

# --- 1. Setup Optimizer, Scheduler, and Loss Function ---

# Optimizer: SGD with Nesterov momentum
optimizer = optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY,
    nesterov=True
)

# Loss Function: Cross-Entropy for multi-class classification
criterion = nn.CrossEntropyLoss()

# Scheduler: 10-epoch linear warmup followed by cosine decay
warmup_epochs = 10
main_epochs = EPOCHS - warmup_epochs

# Define the two schedulers
warmup_scheduler = optim.lr_scheduler.LinearLR(
    optimizer, start_factor=0.01, end_factor=1.0, total_iters=warmup_epochs
)
cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=main_epochs, eta_min=0
)

# Combine them sequentially
scheduler = optim.lr_scheduler.SequentialLR(
    optimizer, schedulers=[warmup_scheduler, cosine_scheduler], milestones=[warmup_epochs]
)

# --- 2. Helper Function for Accuracy Calculation ---
def calculate_accuracy(outputs, labels):
    """Calculates Top-1 and Top-5 accuracy."""
    # Top-1 Accuracy
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    top1_acc = 100 * correct / total
    
    # Top-5 Accuracy
    _, pred_top5 = outputs.topk(5, 1, True, True)
    pred_top5 = pred_top5.t()
    correct_top5 = pred_top5.eq(labels.view(1, -1).expand_as(pred_top5))
    correct_5 = correct_top5[:5].reshape(-1).float().sum(0, keepdim=True)
    top5_acc = 100 * correct_5.item() / total
    
    return top1_acc, top5_acc

# --- 3. Training and Validation Functions ---
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    top1_accuracy = 0.0
    top5_accuracy = 0.0
    
    progress_bar = tqdm(loader, desc="Training", leave=False)
    for joint_batch, bone_batch, labels_batch in progress_bar:
        joint_batch = joint_batch.to(DEVICE)
        bone_batch = bone_batch.to(DEVICE)
        labels_batch = labels_batch.to(DEVICE)
        
        optimizer.zero_grad()
        
        outputs = model(joint_batch, bone_batch)
        loss = criterion(outputs, labels_batch)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        top1_acc, top5_acc = calculate_accuracy(outputs, labels_batch)
        top1_accuracy += top1_acc
        top5_accuracy += top5_acc
        
    avg_loss = running_loss / len(loader)
    avg_top1_acc = top1_accuracy / len(loader)
    avg_top5_acc = top5_accuracy / len(loader)
    return avg_loss, avg_top1_acc, avg_top5_acc

def validate_one_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    top1_accuracy = 0.0
    top5_accuracy = 0.0
    
    with torch.no_grad():
        progress_bar = tqdm(loader, desc="Validating", leave=False)
        for joint_batch, bone_batch, labels_batch in progress_bar:
            joint_batch = joint_batch.to(DEVICE)
            bone_batch = bone_batch.to(DEVICE)
            labels_batch = labels_batch.to(DEVICE)
            
            outputs = model(joint_batch, bone_batch)
            loss = criterion(outputs, labels_batch)
            
            running_loss += loss.item()
            top1_acc, top5_acc = calculate_accuracy(outputs, labels_batch)
            top1_accuracy += top1_acc
            top5_accuracy += top5_acc

    avg_loss = running_loss / len(loader)
    avg_top1_acc = top1_accuracy / len(loader)
    avg_top5_acc = top5_accuracy / len(loader)
    return avg_loss, avg_top1_acc, avg_top5_acc

# --- 4. Main Training Loop ---
best_val_acc = 0.0
print("\n--- Starting Model Training ---")

for epoch in range(EPOCHS):
    start_time = time.time()
    
    train_loss, train_top1, train_top5 = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_top1, val_top5 = validate_one_epoch(model, val_loader, criterion)
    
    # Step the scheduler
    scheduler.step()
    
    epoch_duration = time.time() - start_time
    
    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Time: {epoch_duration:.2f}s | "
        f"LR: {optimizer.param_groups[0]['lr']:.5f} | "
        f"Train Loss: {train_loss:.4f} | Train Top-1: {train_top1:.2f}% | "
        f"Val Loss: {val_loss:.4f} | Val Top-1: {val_top1:.2f}% | Val Top-5: {val_top5:.2f}%"
    )
    
    # Save the best model based on validation Top-1 accuracy
    if val_top1 > best_val_acc:
        best_val_acc = val_top1
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"🎉 New best model saved with Val Top-1 Acc: {best_val_acc:.2f}%")

print("\n--- Training Complete ---")
print(f"Best Validation Top-1 Accuracy: {best_val_acc:.2f}%")
print("The best model has been saved to 'best_model.pth'")


--- Starting Model Training ---


Epoch 1/350 | Time: 35.53s | LR: 0.01090 | Train Loss: 5.6967 | Train Top-1: 0.54% | Val Loss: 5.6769 | Val Top-1: 0.66% | Val Top-5: 1.75%
🎉 New best model saved with Val Top-1 Acc: 0.66%


Epoch 2/350 | Time: 32.97s | LR: 0.02080 | Train Loss: 5.5632 | Train Top-1: 1.13% | Val Loss: 5.3437 | Val Top-1: 1.21% | Val Top-5: 6.14%
🎉 New best model saved with Val Top-1 Acc: 1.21%


Epoch 3/350 | Time: 33.68s | LR: 0.03070 | Train Loss: 5.2827 | Train Top-1: 1.86% | Val Loss: 5.1178 | Val Top-1: 2.63% | Val Top-5: 9.10%
🎉 New best model saved with Val Top-1 Acc: 2.63%


Epoch 4/350 | Time: 33.37s | LR: 0.04060 | Train Loss: 5.1059 | Train Top-1: 2.00% | Val Loss: 4.9622 | Val Top-1: 2.74% | Val Top-5: 12.50%
🎉 New best model saved with Val Top-1 Acc: 2.74%


...

Epoch 346/350 | Time: 33.19s | LR: 0.00003 | Train Loss: 0.0119 | Train Top-1: 99.89% | Val Loss: 1.5442 | Val Top-1: 67.32% | Val Top-5: 87.06%


Epoch 347/350 | Time: 33.35s | LR: 0.00002 | Train Loss: 0.0118 | Train Top-1: 99.92% | Val Loss: 1.5518 | Val Top-1: 67.32% | Val Top-5: 86.73%


Epoch 348/350 | Time: 33.28s | LR: 0.00001 | Train Loss: 0.0123 | Train Top-1: 99.94% | Val Loss: 1.5513 | Val Top-1: 66.89% | Val Top-5: 87.17%


Epoch 349/350 | Time: 33.22s | LR: 0.00000 | Train Loss: 0.0122 | Train Top-1: 99.97% | Val Loss: 1.5481 | Val Top-1: 67.21% | Val Top-5: 87.17%


Epoch 350/350 | Time: 33.25s | LR: 0.00000 | Train Loss: 0.0124 | Train Top-1: 99.97% | Val Loss: 1.5570 | Val Top-1: 67.32% | Val Top-5: 86.95%

--- Training Complete ---
Best Validation Top-1 Accuracy: 67.65%
The best model has been saved to 'best_model.pth'
